In [46]:
# Importing required libraries
from sklearn.model_selection import RepeatedKFold
import catboost
from catboost import datasets
from sklearn.metrics import accuracy_score, confusion_matrix
from bayes_opt import BayesianOptimization
import shap
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier, Pool
import numpy as np
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from catboost import *
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import os
from scipy.stats import spearmanr
from sklearn.metrics import mean_squared_error  # Ensure this import is included
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [44]:
# Set up figures' format
%config InlineBackend.figure_format = 'svg'

In [45]:
# CatBoost check-up
print(catboost.__version__)
!python --version

1.2.3
Python 3.9.18


In [47]:
# Reading input files
ams,preds = pd.read_csv('ams_gsv.csv'), pd.read_csv('ams_gsv_preds.csv')

In [48]:
# Selecting 'income' and 'id' columns
income = ams['income']
id = preds['id']

In [49]:
# Normalising predictor variables
first_col = ams.iloc[:, 0]
remaining_cols = ams.iloc[:, 1:]

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler on the remaining columns of X and transform them
remaining_scaled = pd.DataFrame(scaler.fit_transform(remaining_cols), columns=remaining_cols.columns)

# Reset indices to ensure they align for concatenation
first_col = first_col.reset_index(drop=True)
remaining_scaled = remaining_scaled.reset_index(drop=True)

# Concatenate the first column back with the scaled remaining columns
ams = pd.concat([first_col, remaining_scaled], axis=1)



In [50]:
# Normalising predictor variables
first_col = preds.iloc[:, 0]
remaining_cols = preds.iloc[:, 1:]

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler on the remaining columns of X and transform them
remaining_scaled = pd.DataFrame(scaler.fit_transform(remaining_cols), columns=remaining_cols.columns)

# Reset indices to ensure they align for concatenation
first_col = first_col.reset_index(drop=True)
remaining_scaled = remaining_scaled.reset_index(drop=True)

# Concatenate the first column back with the scaled remaining columns
preds = pd.concat([first_col, remaining_scaled], axis=1)



In [51]:
ams

,income,gsv_apartments,gsv_commercial,gsv_greenery,gsv_historical,gsv_impervious,gsv_industrial,gsv_other,gsv_private,gsv_water,...,gsv_colour_saturation_std,gsv_colour_brightness_mean,gsv_colour_brightness_std,gsv_disorderliness_mean,gsv_disorderliness_std,gsv_std_mean,gsv_std_std,gsv_contrast_mean,gsv_contrast_std,gsv_coherence_std
0,00-20 low,0.000000,0.00,0.000000,0.0,0.00,0.0,0.00,0.000000,0.000000,...,0.172227,0.492173,0.268796,0.862601,0.155033,0.652722,0.273291,0.379061,0.195787,0.205824
1,00-20 low,0.000000,0.00,0.000000,0.0,0.00,0.0,0.00,0.000000,0.000000,...,0.172227,0.492173,0.268796,0.862601,0.155033,0.652722,0.273291,0.379061,0.195787,0.205824
2,00-20 low,0.142857,0.25,0.000000,0.0,0.00,0.0,0.00,0.000000,0.000000,...,0.066005,0.633977,0.318617,0.839201,0.304124,0.681289,0.258832,0.318707,0.243122,0.278741
3,00-20 low,0.142857,0.50,0.000000,0.0,0.00,0.0,0.00,0.000000,0.000000,...,0.162134,0.519754,0.516824,0.899050,0.046900,0.716788,0.272949,0.485765,0.234291,0.042244
4,00-20 low,0.000000,0.00,0.000000,0.0,0.00,0.0,0.00,0.000000,0.000000,...,0.172227,0.492173,0.268796,0.862601,0.155033,0.652722,0.273291,0.379061,0.195787,0.205824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4270,80-100 high,0.000000,0.00,0.333333,0.0,0.25,0.0,0.00,0.000000,0.000000,...,0.152469,0.901167,0.239116,0.642786,0.446232,0.571405,0.165541,0.119023,0.138799,0.121806
4271,80-100 high,0.142857,0.00,0.000000,0.0,0.00,0.0,0.00,0.166667,0.000000,...,0.125397,0.721157,0.323803,0.897157,0.193622,0.796267,0.229601,0.421859,0.242305,0.154616
4272,80-100 high,0.000000,0.00,0.000000,0.0,0.00,0.0,0.25,0.000000,0.333333,...,0.393437,0.446629,0.077663,0.909020,0.163496,0.735336,0.118362,0.311484,0.099330,0.094039
4273,80-100 high,0.000000,0.00,0.000000,0.0,0.00,0.0,0.00,0.000000,0.000000,...,0.172227,0.492173,0.268796,0.862601,0.155033,0.652722,0.273291,0.379061,0.195787,0.205824


In [52]:
# Creating X/y dataframes
y = ams.income
X = preds.drop('id', axis=1)

In [53]:
y

0         00-20 low
1         00-20 low
2         00-20 low
3         00-20 low
4         00-20 low
           ...     
4270    80-100 high
4271    80-100 high
4272    80-100 high
4273    80-100 high
4274    80-100 high
Name: income, Length: 4275, dtype: object

In [54]:
dataset_dir = './inc_ams'
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)

# We will be able to work with files with/without header and
# with different separators.

ams.to_csv(
    os.path.join(dataset_dir, 'train.csv'),
    index=False, sep=',', header=True
)
preds.to_csv(
    os.path.join(dataset_dir, 'preds.csv'),
    index=False, sep=',', header=True
)

In [55]:
pool1 = Pool(data=X, label=y)

print('Dataset shape')
print('dataset 1:' + str(pool1.shape))

print('\n')
print('Column names')
print('dataset 1:')
print(pool1.get_feature_names()) 

Dataset shape
dataset 1:(4275, 21)


Column names
dataset 1:
['gsv_apartments', 'gsv_commercial', 'gsv_greenery', 'gsv_historical', 'gsv_impervious', 'gsv_industrial', 'gsv_other', 'gsv_private', 'gsv_water', 'gsv_colour_hue_mean', 'gsv_colour_hue_std', 'gsv_colour_saturation_std', 'gsv_colour_brightness_mean', 'gsv_colour_brightness_std', 'gsv_disorderliness_mean', 'gsv_disorderliness_std', 'gsv_std_mean', 'gsv_std_std', 'gsv_contrast_mean', 'gsv_contrast_std', 'gsv_coherence_std']


In [56]:
# Train/validation split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.8, random_state=42)

In [57]:
# Hyperparameters optimization
# Define the function to optimize
def catboost_cv(learning_rate, iterations, depth, l2_leaf_reg):
    # Convert continuous parameters to integer where necessary
    iterations = int(iterations)
    depth = int(round(depth))
    l2_leaf_reg = int(round(l2_leaf_reg))
    
    # Initialize KFold
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    train_f1_scores = []
    validation_f1_scores = []
    
    # Perform 5-fold cross-validation
    for train_index, test_index in kf.split(X):
        X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
        y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
        
        train_pool = Pool(data=X_train_fold, label=y_train_fold)
        test_pool = Pool(data=X_test_fold, label=y_test_fold)
        
        # Define and train the model
        model = CatBoostClassifier(
            learning_rate=learning_rate,
            iterations=iterations,
            depth=depth,
            l2_leaf_reg=l2_leaf_reg,
            loss_function='MultiClass',
            eval_metric='TotalF1',
            random_seed=42,
            logging_level='Silent'
        )
        
        model.fit(train_pool, eval_set=test_pool, use_best_model=True)
        
        # Predict on training and validation data
        y_train_pred = model.predict(X_train_fold)
        y_test_pred = model.predict(X_test_fold)
        
        # Calculate F1 scores
        train_f1 = f1_score(y_train_fold, y_train_pred, average='weighted')
        validation_f1 = f1_score(y_test_fold, y_test_pred, average='weighted')
        
        train_f1_scores.append(train_f1)
        validation_f1_scores.append(validation_f1)
    
    # Calculate average F1 scores across all folds
    avg_train_f1 = np.mean(train_f1_scores)
    avg_validation_f1 = np.mean(validation_f1_scores)
    
    # Calculate the difference between training and validation F1 scores
    f1_difference = avg_train_f1 - avg_validation_f1
    
    # Return a very low score if the F1 difference exceeds 0.1
    if f1_difference > 0.1:
        return 0
    
    # Store the results for comparison
    results.append({
        'learning_rate': learning_rate,
        'iterations': iterations,
        'depth': depth,
        'l2_leaf_reg': l2_leaf_reg,
        'avg_train_f1': avg_train_f1,
        'avg_validation_f1': avg_validation_f1,
        'f1_difference': f1_difference
    })
    
    return avg_validation_f1

# Define the initial bounds for hyperparameters
param_bounds = {
    'learning_rate': (0.01, 0.1), 
    'iterations': (500, 2000), 
    'depth': (2, 3), 
    'l2_leaf_reg': (1, 120)
}

# List to store results
results = []

# Initialize Bayesian Optimization
optimizer = BayesianOptimization(
    f=catboost_cv,
    pbounds=param_bounds,
    random_state=63,
    verbose=2
)

# Perform optimization
optimizer.maximize(
    init_points=20,  # Number of initial random points
    n_iter=40       # Number of optimization iterations
)

# Extract the best parameters
best_params = optimizer.max['params']
best_params['iterations'] = int(best_params['iterations'])
best_params['depth'] = int(round(best_params['depth']))
best_params['l2_leaf_reg'] = int(round(best_params['l2_leaf_reg']))

print("Best hyperparameters found were:")
print(best_params)

# Print train/test F1 score comparison
for result in results:
    print(f"Parameters: learning_rate={result['learning_rate']}, iterations={result['iterations']}, depth={result['depth']}, l2_leaf_reg={result['l2_leaf_reg']}")
    print(f"Average Train F1 Score: {result['avg_train_f1']}, Average Validation F1 Score: {result['avg_validation_f1']}, F1 Score Difference: {result['f1_difference']}\n")


|   iter    |  target   |   depth   | iterat... | l2_lea... | learni... |
-------------------------------------------------------------------------
| 1         | 0.0       | 2.554     | 1.094e+03 | 6.727     | 0.05731   |
| 2         | 0.2175    | 2.153     | 1.25e+03  | 31.47     | 0.05888   |
| 3         | 0.0       | 2.548     | 838.7     | 11.62     | 0.05067   |
| 4         | 0.2161    | 2.773     | 1.523e+03 | 103.6     | 0.05085   |
| 5         | 0.2048    | 2.569     | 507.9     | 105.1     | 0.06201   |
| 6         | 0.0       | 2.945     | 1.838e+03 | 82.14     | 0.09013   |
| 7         | 0.0       | 2.193     | 1.704e+03 | 3.786     | 0.09284   |
| 8         | 0.2155    | 2.669     | 1.613e+03 | 115.9     | 0.04598   |
| 9         | 0.2163    | 2.315     | 1.237e+03 | 36.18     | 0.05873   |
| 10        | 0.2028    | 2.293     | 853.8     | 84.25     | 0.04122   |
| 11        | 0.22      | 2.459     | 1.404e+03 | 14.12     | 0.04615   |
| 12        | 0.2018    | 2.262     | 

In [59]:

# Create a CatBoost classifier instance
tuned_model = CatBoostClassifier(
    random_seed=64,
    iterations=1407,
    task_type="CPU",
    learning_rate=0.05841076133949613,
    l2_leaf_reg=49,
    depth=3,
    loss_function='MultiClass'
)

# Fit the model
tuned_model.fit(
    X_train, y_train,
    verbose=False,
    eval_set=(X_validation, y_validation),
    early_stopping_rounds=50,  # Stops if the chosen metric doesn't improve in 50 rounds
    plot=False  # Plots the metric during training
)

# Get results of the evaluation
results = tuned_model.get_evals_result()


In [60]:
# Quality assessment
# Define the model with optimal parameters
tuned_model = CatBoostClassifier(
    random_seed=64,
    iterations=1407,
    task_type="CPU",
    learning_rate=0.05841076133949613,
    l2_leaf_reg=49,
    depth=3,
    loss_function='MultiClass'
)

# Setup Repeated K-Fold cross-validation
rkf = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)

# Prepare lists to store results
train_accuracy_results = []
test_accuracy_results = []
train_f1_results = []
test_f1_results = []
train_auc_results = []
test_auc_results = []

# Loop over each fold
for train_index, test_index in rkf.split(X):
    train_pool = Pool(X.iloc[train_index], y.iloc[train_index])
    test_pool = Pool(X.iloc[test_index], y.iloc[test_index])

    # Fit model
    tuned_model.fit(train_pool, eval_set=test_pool, verbose=False)

    # Predict on train and test sets
    train_predictions = tuned_model.predict(train_pool)
    test_predictions = tuned_model.predict(test_pool)
    
    # Predict probabilities for AUC calculation
    train_probabilities = tuned_model.predict_proba(train_pool)
    test_probabilities = tuned_model.predict_proba(test_pool)

    # Calculate Accuracy
    train_accuracy = accuracy_score(y.iloc[train_index], train_predictions)
    test_accuracy = accuracy_score(y.iloc[test_index], test_predictions)
    train_accuracy_results.append(train_accuracy)
    test_accuracy_results.append(test_accuracy)

    # Calculate F1 Score (macro)
    train_f1 = f1_score(y.iloc[train_index], train_predictions, average='macro')
    test_f1 = f1_score(y.iloc[test_index], test_predictions, average='macro')
    train_f1_results.append(train_f1)
    test_f1_results.append(test_f1)

    # Calculate AUC (macro, multi_class='ovr')
    train_auc = roc_auc_score(y.iloc[train_index], train_probabilities, multi_class='ovr', average='macro')
    test_auc = roc_auc_score(y.iloc[test_index], test_probabilities, multi_class='ovr', average='macro')
    train_auc_results.append(train_auc)
    test_auc_results.append(test_auc)

# Calculate the average and standard deviation of metrics across all train and test folds
mean_train_accuracy = np.mean(train_accuracy_results)
std_train_accuracy = np.std(train_accuracy_results)
mean_test_accuracy = np.mean(test_accuracy_results)
std_test_accuracy = np.std(test_accuracy_results)

mean_train_f1 = np.mean(train_f1_results)
std_train_f1 = np.std(train_f1_results)
mean_test_f1 = np.mean(test_f1_results)
std_test_f1 = np.std(test_f1_results)

mean_train_auc = np.mean(train_auc_results)
std_train_auc = np.std(train_auc_results)
mean_test_auc = np.mean(test_auc_results)
std_test_auc = np.std(test_auc_results)

print("Average Train Accuracy:", mean_train_accuracy)
print("Train Accuracy Standard Deviation:", std_train_accuracy)
print("Average Test Accuracy:", mean_test_accuracy)
print("Test Accuracy Standard Deviation:", std_test_accuracy)
print("Average Train F1 Score:", mean_train_f1)
print("Train F1 Score Standard Deviation:", std_train_f1)
print("Average Test F1 Score:", mean_test_f1)
print("Test F1 Score Standard Deviation:", std_test_f1)
print("Average Train AUC:", mean_train_auc)
print("Train AUC Standard Deviation:", std_train_auc)
print("Average Test AUC:", mean_test_auc)
print("Test AUC Standard Deviation:", std_test_auc)


Average Train Accuracy: 0.345906432748538
Train Accuracy Standard Deviation: 0.023182297576416728
Average Test Accuracy: 0.304327485380117
Test Accuracy Standard Deviation: 0.011974841487628589
Average Train F1 Score: 0.19250908708645878
Train F1 Score Standard Deviation: 0.03521282858943335
Average Test F1 Score: 0.15114081900392243
Test F1 Score Standard Deviation: 0.013036339761004967
Average Train AUC: 0.6280371747659064
Train AUC Standard Deviation: 0.02495290799893737
Average Test AUC: 0.5459801120988257
Test AUC Standard Deviation: 0.011611691943695608


In [62]:
# Confusion matrix
class_names = ['00-20 low', '20-40 below average', '40-60 average', '60-80 above average', '80-100 high']

# Ensure 'y' is a single-column DataFrame and convert it to a Series
if isinstance(y, pd.DataFrame):
    y = y.iloc[:, 0]

# Make predictions using the CatBoost model
predicted_values = tuned_model.predict(X)

# Calculate accuracy
accuracy = accuracy_score(y, predicted_values) * 100

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y, predicted_values, labels=class_names)

# Transpose the confusion matrix
conf_matrix = conf_matrix.T

# Plot confusion matrix using ConfusionMatrixDisplay
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=class_names)
fig, ax = plt.subplots(figsize=(6, 6))  # Create a figure and axis
disp.plot(cmap='Blues', xticks_rotation='vertical', ax=ax)
ax.set_title(f'\nAccuracy: {accuracy:.2f}%')
ax.set_xlabel('Actual income classes in Amsterdam')
ax.set_ylabel('Predicted income classes in Amsterdam')

# Save the plot as an SVG file
plt.savefig('confusion_matrix_ams_gsv.svg', format='svg')

# Show the plot
plt.show()


In [ ]:
# Extracting SHAP values
if isinstance(y, pd.DataFrame):
    y = y.iloc[:, 0]

# Convert 'y' to a numpy array
y = np.array(y)

# Initialize the SHAP explainer
explainer = shap.TreeExplainer(tuned_model)

# Calculate SHAP values for the dataset
shap_values = explainer.shap_values(X)

# For categorical targets, SHAP returns a list of arrays (one for each class)
# We will focus on the class '80-100 high'
target_class_index = class_names.index('80-100 high')
shap_values_for_class = shap_values[target_class_index]

# Ensure shap_values_for_class is a DataFrame
shap_df = pd.DataFrame(shap_values_for_class, columns=X.columns)

# Extract SHAP values for the specific predictor 'colour_hue_disorderliness_mean'
shap_tcap_wetness_mean = shap_df['tcap_wetness_mean']
shap_colour_brightness_coherence_mean = shap_df['colour_brightness_coherence_mean']
shap_tcap_wetness_disorderliness_mean = shap_df['tcap_wetness_disorderliness_mean']
shap_surface_roughness_disorderliness_mean = shap_df['surface_roughness_disorderliness_mean']
shap_ndwi_wetness_contrast_mean = shap_df['ndwi_wetness_contrast_mean']
shap_ndwi_wetness_coherence_std = shap_df['ndwi_wetness_coherence_std']
# Ensure all arrays are 1-dimensional
y = y.flatten()
predicted_values = tuned_model.predict(X).flatten()
shap_tcap_wetness_mean = shap_tcap_wetness_mean.values.flatten()
shap_colour_brightness_coherence_mean = shap_colour_brightness_coherence_mean.values.flatten()
shap_tcap_wetness_disorderliness_mean = shap_tcap_wetness_disorderliness_mean.values.flatten()
shap_surface_roughness_disorderliness_mean = shap_surface_roughness_disorderliness_mean.values.flatten()
shap_ndwi_wetness_contrast_mean = shap_ndwi_wetness_contrast_mean.values.flatten()
shap_ndwi_wetness_coherence_std = shap_ndwi_wetness_coherence_std.values.flatten()

# Create a DataFrame with actual values, predicted values, and SHAP values for 'colour_hue_disorderliness_mean'
results_with_shap_df = pd.DataFrame({
    'Actual': y,
    'Predicted': predicted_values,
    'SHAP_tcap_wetness_mean': shap_tcap_wetness_mean,
    'SHAP_colour_brightness_coherence_mean': shap_colour_brightness_coherence_mean,
    'SHAP_tcap_wetness_disorderliness_mean': shap_tcap_wetness_disorderliness_mean,
    'SHAP_surface_roughness_disorderliness_mean': shap_surface_roughness_disorderliness_mean,
    'SHAP_ndwi_wetness_contrast_mean': shap_ndwi_wetness_contrast_mean,
    'SHAP_ndwi_wetness_coherence_std': shap_ndwi_wetness_coherence_std
})

# Print the DataFrame
print(results_with_shap_df.head())


In [ ]:
combined_df = results_with_shap_df.join(income, how='inner')  # Use 'inner' join to keep only matching indices
combined_df

In [ ]:
combined_df = combined_df.join(id, how='inner')  # Use 'inner' join to keep only matching indices
combined_df

In [ ]:
# Errors assessment
# Define the categories and their order
categories = ['00-20 low', '20-40 below average', '40-60 average', '60-80 above average', '80-100 high']

# Define the function to calculate the error
def calculate_error(actual, predicted, categories):
    actual_index = categories.index(actual)
    predicted_index = categories.index(predicted)
    difference = predicted_index - actual_index
    
    if difference == 0:
        return 'accurate'
    elif difference > 0:
        return f'overestimated {difference} class{"es" if difference > 1 else ""}'
    else:
        return f'underestimated {abs(difference)} class{"es" if abs(difference) > 1 else ""}'

# Apply the function to each row in the DataFrame
combined_df['Error'] = combined_df.apply(lambda row: calculate_error(row['Actual'], row['Predicted'], categories), axis=1)

# Display the DataFrame with the new column
combined_df

In [ ]:
combined_df.to_csv('pred_vs_actual_with_shap_and_id_ams.csv', index=False)

In [63]:
# Plotting iterations
epochs = np.arange(len(results['learn']['MultiClass']))

# Retrieve the best iteration from the model
best_iteration = tuned_model.get_best_iteration()

plt.figure(figsize=(10, 5))
plt.plot(epochs, results['learn']['MultiClass'], label='Train')
plt.plot(epochs, results['validation']['MultiClass'], label='Validation')

# Mark the best iteration on the plot
if best_iteration is not None:
    plt.scatter(best_iteration, results['validation']['MultiClass'][best_iteration], color='red', zorder=5, label='Best iteration')

plt.xlabel('Epochs')
plt.ylabel('MultiClass')
plt.title('Learning curve')
plt.legend()
plt.show()


In [64]:
pool1 = Pool(data=X, label=y)

print('Dataset shape')
print('dataset 1:' + str(pool1.shape))

print('\n')
print('Column names')
print('dataset 1:')
print(pool1.get_feature_names()) 

Dataset shape
dataset 1:(4275, 21)


Column names
dataset 1:
['gsv_apartments', 'gsv_commercial', 'gsv_greenery', 'gsv_historical', 'gsv_impervious', 'gsv_industrial', 'gsv_other', 'gsv_private', 'gsv_water', 'gsv_colour_hue_mean', 'gsv_colour_hue_std', 'gsv_colour_saturation_std', 'gsv_colour_brightness_mean', 'gsv_colour_brightness_std', 'gsv_disorderliness_mean', 'gsv_disorderliness_std', 'gsv_std_mean', 'gsv_std_std', 'gsv_contrast_mean', 'gsv_contrast_std', 'gsv_coherence_std']


In [65]:
metrics = tuned_model.eval_metrics(
    data=pool1,
    metrics=['TotalF1','AUC', 'Accuracy'],
    ntree_start=0,
    ntree_end=0,
    eval_period=1,
    plot=False
)

In [66]:
# Extracting metrics for plotting
rmse_values = metrics['TotalF1']
r2_values = metrics['Accuracy']
trees = list(range(len(rmse_values)))  # Assuming eval_period=1 and starting from the first tree

# Creating subplots
fig, axs = plt.subplots(2, 1, figsize=(10, 10))

# Plotting RMSE
axs[0].plot(trees, rmse_values, label='TotalF1', color='tab:blue')
axs[0].set_title('Total F1 over trees')
axs[0].set_xlabel('Number of trees')
axs[0].set_ylabel('Total F1')
axs[0].legend()

# Plotting R2
axs[1].plot(trees, r2_values, label='Accuracy', color='tab:orange')
axs[1].set_title('Accuracy over trees')
axs[1].set_xlabel('Number of trees')
axs[1].set_ylabel('Accuracy')
axs[1].legend()

plt.tight_layout()
plt.show()


In [67]:
tuned_model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,gsv_contrast_mean,10.434862
1,gsv_colour_brightness_std,9.102764
2,gsv_apartments,9.094698
3,gsv_std_std,8.467292
4,gsv_colour_hue_mean,8.047206
5,gsv_std_mean,7.711084
6,gsv_coherence_std,6.749240
7,gsv_disorderliness_mean,5.680134
8,gsv_historical,5.335509
9,gsv_colour_brightness_mean,4.561757


In [68]:

# Confusion matrix
class_names = ['00-20 low', '20-40 below average', '40-60 average', '60-80 above average', '80-100 high']

# Plotting the confusion matrix
disp = ConfusionMatrixDisplay.from_estimator(
    tuned_model,
    X,
    y,
    display_labels=class_names,
    xticks_rotation='vertical'
)

plt.show()


In [69]:
params = tuned_model.get_params()
params['loss_function'] = 'MultiClass'
params['custom_loss'] = 'TotalF1'


cv_data = cv(
    params = params,
    pool = Pool(X, label=y),
    fold_count=5,
    shuffle=True,
    partition_random_seed=0,
    plot=False,
    stratified=False,
    verbose=False
)

Training on fold [0/5]

bestTest = 1.475548605
bestIteration = 350

Training on fold [1/5]

bestTest = 1.496062845
bestIteration = 300

Training on fold [2/5]

bestTest = 1.454625207
bestIteration = 222

Training on fold [3/5]

bestTest = 1.498138966
bestIteration = 631

Training on fold [4/5]

bestTest = 1.511530531
bestIteration = 714



In [70]:
cv_data

,iterations,test-MultiClass-mean,test-MultiClass-std,train-MultiClass-mean,train-MultiClass-std,test-TotalF1-mean,test-TotalF1-std,train-TotalF1-mean,train-TotalF1-std
0,0,1.600852,0.000784,1.600657,0.000515,0.133878,0.020429,0.138622,0.008994
1,1,1.592580,0.001549,1.592196,0.000975,0.154695,0.019305,0.156242,0.004438
2,2,1.585863,0.002598,1.585264,0.001254,0.151880,0.018658,0.159064,0.010856
3,3,1.579695,0.003383,1.578838,0.001607,0.160770,0.022186,0.169544,0.016882
4,4,1.574342,0.004045,1.573253,0.001951,0.164742,0.029075,0.171694,0.016189
...,...,...,...,...,...,...,...,...,...
1402,1402,1.494093,0.020228,1.372938,0.005228,0.219941,0.010099,0.343258,0.005651
1403,1403,1.494099,0.020232,1.372889,0.005235,0.219894,0.009972,0.343030,0.005415
1404,1404,1.494108,0.020259,1.372805,0.005260,0.219984,0.009832,0.343324,0.005154
1405,1405,1.494148,0.020271,1.372753,0.005282,0.220010,0.009792,0.343399,0.004973


In [71]:
# Extracting the mean RMSE and R2 values from cv_data
mean_multiclass = cv_data['test-MultiClass-mean']
mean_f1 = cv_data['test-TotalF1-mean']
iterations = range(len(mean_multiclass))

# Plotting the RMSE and R2 values
fig, ax1 = plt.subplots()

ax1.set_xlabel('Iterations')
ax1.set_ylabel('MultiClass', color='tab:blue')
ax1.plot(iterations, mean_multiclass, label='MultiClass', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Instantiate a second axes that shares the same x-axis
ax2 = ax1.twinx()
ax2.set_ylabel('TotalF1', color='tab:orange')
ax2.plot(iterations, mean_f1, label='TotalF1', color='tab:orange')
ax2.tick_params(axis='y', labelcolor='tab:orange')

# Title and legend
plt.title('CatBoost CV metrics over iterations')
fig.tight_layout()  # To ensure the right y-label is not clipped
plt.show()

In [72]:
multiclass_values = cv_data['train-MultiClass-mean']
f1_values = cv_data['train-TotalF1-mean']

# Create a DataFrame with both RMSE and R2 values
cv_results_df = pd.DataFrame({'MultiClass': multiclass_values, 'TotalF1': f1_values})
cv_results_df

,MultiClass,TotalF1
0,1.600657,0.138622
1,1.592196,0.156242
2,1.585264,0.159064
3,1.578838,0.169544
4,1.573253,0.171694
...,...,...
1402,1.372938,0.343258
1403,1.372889,0.343030
1404,1.372805,0.343324
1405,1.372753,0.343399


In [73]:
multiclass_values = cv_data['test-MultiClass-mean']
f1_values = cv_data['test-TotalF1-mean']

# Create a DataFrame with both RMSE and R2 values
cv_results_df = pd.DataFrame({'MultiClass': multiclass_values, 'TotalF1': f1_values})
cv_results_df

,MultiClass,TotalF1
0,1.600852,0.133878
1,1.592580,0.154695
2,1.585863,0.151880
3,1.579695,0.160770
4,1.574342,0.164742
...,...,...
1402,1.494093,0.219941
1403,1.494099,0.219894
1404,1.494108,0.219984
1405,1.494148,0.220010


In [75]:
shap_values = tuned_model.get_feature_importance(pool1, fstr_type='ShapValues')
print(shap_values.shape)

(4275, 5, 22)


In [76]:
explainer = shap.TreeExplainer(tuned_model)
shap_values = explainer.shap_values(Pool(X, y))

#shap.initjs()
#shap.force_plot(explainer.expected_value[0],shap_values[0])

In [77]:
shap.summary_plot(shap_values, X.values, plot_type="bar", class_names= class_names, feature_names = X.columns)
plt.title('The Summary Plot for the Multiclass Model'+'\n'+'Class 2 - Best, Class 1 - Premium, Class 0 - Value')

Text(0.5, 1.0, 'The Summary Plot for the Multiclass Model\nClass 2 - Best, Class 1 - Premium, Class 0 - Value')

In [78]:
from sklearn.preprocessing import LabelEncoder

labels = [
    "00-20 low",
    "20-40 below average",
    "40-60 average",
    "60-80 above average",
    "80-100 high",
]
le = LabelEncoder()
y = le.fit_transform(labels)
encoding_scheme = dict(zip(y, labels))
print(encoding_scheme)

{0: '00-20 low', 1: '20-40 below average', 2: '40-60 average', 3: '60-80 above average', 4: '80-100 high'}


In [79]:
plt.title('Feature importances for Amsterdam')
shap.summary_plot(shap_values, X.values, plot_type="bar", class_names= class_names, feature_names = X.columns)

In [80]:
# Dependency plot for low-income class
plt.switch_backend('Agg')

# Set the title of the plot
plt.title('Impact on low income in Amsterdam')

# Generate SHAP summary plot
shap.summary_plot(shap_values[0], X.values, feature_names=X.columns, show=False)  # Set show=False to prevent immediate display

# After generating the plot, get the current figure
fig = plt.gcf()

# Save the current figure in SVG format
fig.savefig("shap_dependency_plot_lowAMS_gsv.svg", format='svg', bbox_inches='tight')

# Optionally, close the figure to free up memory
plt.close(fig)


Auto-close()ing of figures upon backend switching is deprecated since 3.8 and will be removed two minor releases later.  To suppress this warning, explicitly call plt.close('all') first.


In [81]:
plt.title('Impact on income below average in Amsterdam')
shap.summary_plot(shap_values[1], X.values, feature_names = X.columns)

In [82]:
plt.title('Impact on average income in Amsterdam')
shap.summary_plot(shap_values[2], X.values, feature_names = X.columns)

In [83]:
plt.title('Impact on income above average in Amsterdam')
shap.summary_plot(shap_values[3], X.values, feature_names = X.columns)

In [84]:
# Dependency plot for high-income class
# Switch to 'Agg' backend, which is better suited for file outputs
plt.switch_backend('Agg')

# Set the title of the plot
plt.title('Impact on high income in Amsterdam')

# Generate SHAP summary plot
shap.summary_plot(shap_values[4], X.values, feature_names=X.columns, show=False)  # Set show=False to prevent immediate display

# After generating the plot, get the current figure
fig = plt.gcf()

# Save the current figure in SVG format
fig.savefig("shap_dependency_plot_highAMS_gsv.svg", format='svg', bbox_inches='tight')

# Optionally, close the figure to free up memory
plt.close(fig)


Auto-close()ing of figures upon backend switching is deprecated since 3.8 and will be removed two minor releases later.  To suppress this warning, explicitly call plt.close('all') first.
